# Live demo - Anthony

## Importaciones necesarias

In [1]:
import torch
import torchvision
import cv2
import numpy as np

## Se prepara el modelo (esta vez no pre-entrenado)

In [2]:
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model = model.cuda().eval().half()

## Se carga el modelo que se entrenó

In [3]:
model.load_state_dict(torch.load('best_model_resnet18.pth'))

## Se define 'cuda' para que no se quede sin ejecutar en la GPU

In [4]:
device = torch.device('cuda')

## Se optienen las librerías de TensorRT para optimizar el modelo, con esto se crea un modelo nuevo que será utilizado en el live demo (puede demorar)

In [5]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

In [6]:
torch.save(model_trt.state_dict(), 'best_model_trt.pth')

## Se carga el modelo optimizado

---

Se convierten las imágenes de la cámara al mismo formato que las necesita el modelo. Para esto, convertimos de diseño HWC a diseño CHW.

Transferimos los datos del CPU al GPU y añadimos la dimensión del batch

In [7]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_model_trt.pth'))

In [8]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

## Mostramos la cámara

---

Se muestra la cámara para ver lo que el robot está viendo. Se instancia el robot para controlar sus motores.

In [9]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

In [10]:
from jetbot import Robot

robot = Robot()

## Configurar la fuerza de los motores

---

Esto resulta necesario por la desventaja que tiene uno de los motores de Anthony, por lo que se requiere configurar de diferente manera la fuerza de los motores. 

La configuración fue:
> motor izquierdo : 2.5

> motor derecho : 1.5

In [11]:
robot.left_motor.alpha =2.5
robot.right_motor.alpha = 1.5

## Función que le permitirá al robot observar cada cambio en la cámara

---

Esta función realiza los siguientes pasos:


1.   Pre procesa la imagen recibida
2.   Ejecuta la red neuronal
3.   Calcula el valor de la dirección
4.   Controla los motores


In [12]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model_trt(x)
    #print(y)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    #print(y)
    
    prob_blocked = float(y.flatten()[0])
    #print(prob_blocked)
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(0.20)
    else:
        robot.left(0.19)
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

## Se agrega el observador a la cámara para que esta llame la función con vada cambio

In [13]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In [14]:
camera.stop() # se detiene la cámara 

## Se detiene el observador y el robot

In [15]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()